In [45]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')
from typing import List, Dict

# Load data
train_path = r'C:\Users\40108679\OneDrive - Anheuser-Busch InBev\Desktop\Hackathon-2025\lte_participants_data\BG_Data_Hackathon_Train_masked.csv'
test_path = r'C:\Users\40108679\OneDrive - Anheuser-Busch InBev\Desktop\Hackathon-2025\lte_participants_data\BG_Data_Hackathon_Test_Predict_masked.csv'

print("Loading data files...")
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Standardize column names
if 'Year' in df_test.columns:
    df_test = df_test.rename(columns={'Year': 'year', 'Quarter': 'quarter', 'Country': 'country', 
                                     'Brand': 'brand', 'Predicted Power': 'predicted_power'})

# Create quarter numeric values for time series ordering
df_train['quarter_num'] = df_train['quarter'].str.replace('Qtr', '').astype(int)
df_test['quarter_num'] = df_test['quarter'].str.replace('Qtr', '').astype(int)

# Create time index for easier analysis
df_train['time_idx'] = df_train['year']*10 + df_train['quarter_num']
df_test['time_idx'] = df_test['year']*10 + df_test['quarter_num']

# Create a combined DataFrame for easier sorting and mapping
df_test['is_test'] = True
df_train['is_test'] = False
combined = pd.concat([df_train, df_test], ignore_index=True)
combined = combined.sort_values(['brand', 'country', 'year', 'quarter_num'])

print(f"Training data: {len(df_train)} rows")
print(f"Test data: {len(df_test)} rows")

# Estimate brand metrics for test data using brand and country averages
def estimate_brand_metrics():
    print("Estimating brand metrics for test data...")
    
    # Calculate averages by brand and country
    brand_avgs = df_train.groupby('brand').agg({
        'meaning': 'mean',
        'difference': 'mean',
        'salience': 'mean',
        'premium': 'mean'
    })
    
    country_avgs = df_train.groupby('country').agg({
        'meaning': 'mean',
        'difference': 'mean',
        'salience': 'mean',
        'premium': 'mean'
    })
    
    # Global averages (fallback)
    global_avgs = {
        'meaning': df_train['meaning'].mean(),
        'difference': df_train['difference'].mean(),
        'salience': df_train['salience'].mean(),
        'premium': df_train['premium'].mean()
    }
    
    # Fill in metrics for each test row
    metrics = ['meaning', 'difference', 'salience', 'premium']
    for idx, row in df_test.iterrows():
        brand, country = row['brand'], row['country']
        
        # Try brand average first
        if brand in brand_avgs.index:
            for metric in metrics:
                df_test.loc[idx, metric] = brand_avgs.loc[brand, metric]
        
        # Then try country average
        elif country in country_avgs.index:
            for metric in metrics:
                df_test.loc[idx, metric] = country_avgs.loc[country, metric]
        
        # Fallback to global average
        else:
            for metric in metrics:
                df_test.loc[idx, metric] = global_avgs[metric]
    
    return df_test

# Apply estimation to get test metrics
df_test = estimate_brand_metrics()
print(df_test)



Loading data files...
Training data: 1622 rows
Test data: 104 rows
Estimating brand metrics for test data...
     year quarter     country       brand  predicted_power  quarter_num  \
0    2024    Qtr3  a19bbfeafb  cd46a210c6              NaN            3   
1    2024    Qtr3  a19bbfeafb  b6a118bf97              NaN            3   
2    2024    Qtr3  a19bbfeafb  53a67488ad              NaN            3   
3    2024    Qtr3  a19bbfeafb  a0bc54a71d              NaN            3   
4    2024    Qtr3  a19bbfeafb  85498197e2              NaN            3   
..    ...     ...         ...         ...              ...          ...   
99   2025    Qtr2  aa30935544  96ddb9088d              NaN            2   
100  2025    Qtr2  aa30935544  6e31ce004a              NaN            2   
101  2025    Qtr2  aa30935544  71e9e5f0c0              NaN            2   
102  2025    Qtr2  aa30935544  a5f7e77993              NaN            2   
103  2025    Qtr2  aa30935544  e7286b8344              NaN        

In [46]:
df_test.head()

,year,quarter,country,brand,predicted_power,quarter_num,time_idx,is_test,meaning,difference,salience,premium
0,2024,Qtr3,a19bbfeafb,cd46a210c6,NaN,3,20243,True,140.885669,113.861601,160.464473,1.155262
1,2024,Qtr3,a19bbfeafb,b6a118bf97,NaN,3,20243,True,124.081871,104.935099,120.073537,1.054426
2,2024,Qtr3,a19bbfeafb,53a67488ad,NaN,3,20243,True,86.743503,75.443914,79.131779,0.901318
3,2024,Qtr3,a19bbfeafb,a0bc54a71d,NaN,3,20243,True,116.876435,119.421693,81.785540,1.128386
4,2024,Qtr3,a19bbfeafb,85498197e2,NaN,3,20243,True,149.631097,157.905315,125.040083,1.290739


In [47]:
df_train.head()

,year,quarter,country,brand,meaning,difference,salience,power,premium,quarter_num,time_idx,is_test
0,2022,Qtr1,653b4e6d12,de903bdf13,130.254295,105.026180,95.584143,3.075272,1.144869,1,20221,False
1,2022,Qtr2,653b4e6d12,de903bdf13,127.584560,103.211433,97.783428,3.247745,1.135222,2,20222,False
2,2022,Qtr3,653b4e6d12,de903bdf13,130.226966,103.829670,95.621699,3.159509,1.154649,3,20223,False
3,2022,Qtr4,653b4e6d12,de903bdf13,134.213403,103.584346,97.724214,3.428511,1.172568,4,20224,False
4,2023,Qtr1,653b4e6d12,de903bdf13,130.779928,103.717619,101.117547,3.590078,1.164770,1,20231,False


In [48]:
def calculate_iqr_bounds(df: pd.DataFrame, columns: List[str]) -> dict:
    """
    Calculates the IQR bounds (upper and lower) for specified columns in the DataFrame.
    :param df: DataFrame to calculate IQR bounds for
    :param columns: List of column names to calculate bounds for
    :return: Dictionary with column names as keys and (lower_bound, upper_bound) as values
    """
    # Check if all specified columns are present in the DataFrame
    missing_columns = [col for col in columns if col not in df.columns]
    if missing_columns:
        raise ValueError(
            f"The following columns are not in the DataFrame: {missing_columns}"
        )

    bounds = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        bounds[col] = (lower_bound, upper_bound)

    return bounds


def cap_outliers(df: pd.DataFrame, bounds: dict) -> pd.DataFrame:
    df_capped = df.copy()
    for col, (low, high) in bounds.items():
        df_capped[col] = df_capped[col].clip(lower=low, upper=high)
    return df_capped

In [ ]:
# Import scikit-learn libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Prepare data for model training
print("Preparing data for prediction model...")

# Only use train data with non-null 'power'
train_data = df_train[df_train['power'].notnull()].copy()

# apply outlier capping
numeric_cols = ['meaning', 'difference', 'salience', 'premium', 'power']
iqr_bounds = calculate_iqr_bounds(train_data, numeric_cols)
train_data = cap_outliers(train_data, iqr_bounds)




# Encode categorical features
label_encoders = {}
for col in ['country', 'brand']:
    le = LabelEncoder()
    train_data[f'{col}_encoded'] = le.fit_transform(train_data[col])
    df_test[f'{col}_encoded'] = le.transform(df_test[col])
    label_encoders[col] = le

# Select features and target
features = ['year', 'quarter_num', 'country_encoded', 'brand_encoded', 
            'meaning', 'difference', 'salience', 'premium']
target = 'power'

# Split data for validation
X_train, X_val, y_train, y_val = train_test_split(
    train_data[features], 
    train_data[target], 
    test_size=0.2, 
    random_state=42
)

# Train Random Forest model

model = Ridge(alpha=1.0)

model.fit(X_train, y_train)

# Evaluate on validation set
val_pred = model.predict(X_val)
val_score = r2_score(y_val, val_pred)
print(f"Model R² score on validation data: {val_score:.4f}")

# Predict on test set
print("Making predictions on test set...")
df_test['power'] = model.predict(df_test[features])

# Apply minimum value constraint (power cannot be negative)
df_test['power'] = df_test['power'].clip(lower=0)

# Show predictions
print("\nSample predictions:")
print(df_test[['year', 'quarter', 'country', 'brand', 'power']].head(10))

# Save predictions to CSV
submission = df_test[['year', 'quarter', 'country', 'brand', 'power']]
submission.to_csv('brand_power_predictions.csv', index=False)
print("\nPredictions saved to 'brand_power_predictions.csv'")

Preparing data for prediction model...


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- power


In [44]:
# Add this cell to calculate evaluation metrics for your predictions

import numpy as np
from sklearn.metrics import mean_squared_error

def calculate_pps():
    print("Calculating Power Prediction Score (PPS)...")
    
    # We need to compare against validation data since we don't have true values for test set
    # Use the validation split you already created
    y_true = y_val
    y_pred = val_pred
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Calculate standard deviation of target variable
    sigma_y = np.std(y_true)
    
    # Calculate RMSE Skill Score
    rmse_skill = max(0, min(1, 1 - rmse/sigma_y))
    print(f"RMSE: {rmse:.4f}")
    print(f"Standard Deviation of Power: {sigma_y:.4f}")
    print(f"RMSE Skill Score: {rmse_skill:.4f}")
    
    # Calculate Trend Hit Rate
    # Group data by brand and country
    validation_data = X_val.copy()
    validation_data['power_true'] = y_true
    validation_data['power_pred'] = y_pred
    
    # Add time_idx back to validation data for sorting
    # Create time index from year and quarter_num which are already in X_val
    validation_data['time_idx'] = validation_data['year']*10 + validation_data['quarter_num']
    
    # Add brand and country back to validation data for grouping
    validation_data['brand'] = train_data.loc[validation_data.index, 'brand'].values
    validation_data['country'] = train_data.loc[validation_data.index, 'country'].values
    
    # Calculate trend hit rate across brand-country pairs
    trend_hits = 0
    total_cases = 0
    
    # Group by brand and country
    for (brand, country), group in validation_data.groupby(['brand', 'country']):
        # Sort by time
        group = group.sort_values('time_idx')
        
        # Need at least 2 points to calculate trend
        if len(group) < 2:
            continue
            
        # Calculate deltas for true and predicted values
        true_deltas = np.sign(np.diff(group['power_true']))
        pred_deltas = np.sign(np.diff(group['power_pred']))
        
        # Compare trends (direction of change)
        hits = (true_deltas == pred_deltas).sum()
        
        trend_hits += hits
        total_cases += len(true_deltas)
    
    # Calculate trend hit rate
    trend_hit_rate = trend_hits / total_cases if total_cases > 0 else 0
    print(f"Trend Hits: {trend_hits} out of {total_cases}")
    print(f"Trend Hit Rate: {trend_hit_rate:.4f}")
    
    # Calculate final PPS
    pps = 0.5 * rmse_skill + 0.5 * trend_hit_rate
    print(f"Power Prediction Score (PPS): {pps:.4f}")
    
    return pps, rmse_skill, trend_hit_rate

# Calculate PPS
pps, rmse_skill, trend_hit_rate = calculate_pps()

Calculating Power Prediction Score (PPS)...
RMSE: 0.3443
Standard Deviation of Power: 1.6648
RMSE Skill Score: 0.7932
Trend Hits: 162 out of 214
Trend Hit Rate: 0.7570
Power Prediction Score (PPS): 0.7751
